<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BUILDING-A-CLASSIFIER-ON-TOP-OF-THE-PRETRAINED-MODEL" data-toc-modified-id="BUILDING-A-CLASSIFIER-ON-TOP-OF-THE-PRETRAINED-MODEL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BUILDING A CLASSIFIER ON TOP OF THE PRETRAINED MODEL</a></span></li></ul></div>

## BUILDING A CLASSIFIER ON TOP OF THE PRETRAINED MODEL

In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
cov_base=ResNet50(include_top=False,input_shape=(250,250,3))

In [3]:
cov_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
net=keras.Sequential([
    cov_base,
    layers.Flatten(),
    layers.Dense(24,activation='relu'),
    layers.Dropout(rate=0.4),
    layers.Dense(1,activation='sigmoid')
])

Before WE compile and train the model, it’s important to freeze the convolutional base SINCE we want to use it parameters without updating them. <b>Freezing a layer or set of layers means preventing their weights from being updated during training</b>.  In Keras, you freeze a network by setting its trainable attribute to False:

In [5]:
print('The number of trainable weights before freezing the conv base:',
     len(net.trainable_weights))

The number of trainable weights before freezing the conv base: 216


In [6]:
cov_base.trainable=False

In [7]:
print('The number of trainable weights after freezing the conv base:',
     len(net.trainable_weights))

The number of trainable weights after freezing the conv base: 4


<b>With conv_base.trainable=False, only the weights of the fully connected layers build on top of cov_base will be updated.</b>

In [8]:
net.compile(optimizer=keras.optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

In [9]:
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 24)                3145752   
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 26,733,489
Trainable params: 3,145,777
Non-trainable params: 23,587,712
_________________________________________________________________
